In [4]:
import pandas as pd
from collections import Counter
output_path = "../data/processed/processed_parquet.parquet"

In [2]:
data = pd.read_excel('../data/raw/online_retail.xlsx')
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [7]:
sampled_data = data.groupby('Country', group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))

In [8]:
sampled_data['InvoiceNo'] = sampled_data['InvoiceNo'].astype('str')
sampled_data['StockCode'] = sampled_data['StockCode'].astype('str')
sampled_data['Description'] = sampled_data['Description'].astype('str')

In [ ]:
categories = {
    "Home & Kitchen": [
        "CUP", "MUG", "TEA", "COFFEE", "DINNERWARE", "KITCHEN", "NAPKIN", "PLATE", "CUTLERY", 
        "BOTTLE", "SPOON", "FORK", "TINS", "JAR", "BASKET", "STORAGE", "APPLE CORER", "PEELER", 
        "SLICER", "GRATER", "TOWEL", "COASTER", "APRONS", "TRAY", "TONGS", "SQUEEZER", 
        "FRYING PAN", "TIN", "JAM", "BEAKER", "BOX", "DRAWER", "CABINET", "RACK", "STAND", "TRINKET"
    ],
    "Decor & Accessories": [
        "DECORATION", "HEART", "LIGHT", "HOLDER", "VASE", "ORNAMENT", "CANDLE", "POSTER", 
        "FRAME", "PAINTING", "SCULPTURE", "CLOCK", "WALL ART", "FLYING DUCKS", "PANTRY DESIGN", 
        "LANTERN", "STATUE", "METAL", "SIGN", "RETROSPOT", "VINTAGE", "DOOR", "HOOK", "BOARD", 
        "WOOD", "SLATE", "PAISLEY", "IVORY", "CUSHION", "HOME", "FLORAL", "POLKADOT", "ROSE", 
        "JUMBO", "TREASURE", "VICTORIAN", "EDWARDIAN", "GLASS", "ACRYLIC"
    ],
    "Toys, Games & Stationery": [
        "TOY", "GAME", "PUZZLE", "DOLL", "PLUSH", "BOARD GAME", "DOMINOES", "JIGSAW", "PAPER", 
        "NOTEBOOK", "PEN", "STATIONERY", "CARD", "STICKER", "PLANNER", "ENVELOPE", "FOLDER", 
        "BINDER", "JOURNAL", "SKETCHBOOK", "MAGNET", "WOODEN RULER", "HANDMADE", "CRAFTED", 
        "ARTISAN", "EMBROIDERY", "CERAMIC", "BOOK", "MARKER"
    ],
    "Fashion & Accessories": [
        "RAIN MAC", "RAINCOAT", "PONCHO", "SCARF", "GLOVES", "HAT", "SOCKS", "BANDANA", "BELT", 
        "JACKET", "HANDBAG", "SHOES", "EARRING", "NECKLACE", "BRACELET", "COAT", "FELT", "SEWING", "FELTCRAFT"
    ],
    "Gift Wrap & Packaging": [
        "WRAP", "GIFT WRAP", "GIFT", "RIBBON", "SURPRISE", "PEG PLACE", "BUNDLE", "SET OF", "PACK", 
        "TISSUES", "TAPE", "BOW", "TAG", "POSTAGE", "GARLAND"
    ],
    "Outdoor & Garden": [
        "GARDEN", "PLANT", "POTS", "SEED", "FLOWER", "WATERING", "FENCING", "SOIL", "OUTDOOR", 
        "PLANTER", "LAWN", "BIRDHOUSE", "WINDCHIME", "STATUE", "SANDCASTLE", "FLAGS"
    ],
    "Holidays & Seasonal": [
        "CHRISTMAS", "XMAS", "TREE", "FESTIVE", "SANTA", "EASTER", "EGG", "HOLIDAY", "WREATH", 
        "SLEIGH BELLS", "HALLOWEEN", "THANKSGIVING", "NEW YEAR", "BALLOON", "PARTY", "CONFETTI", 
        "BANNER", "BUNTING"
    ],
    "Health & Beauty": [
        "SOAP", "LOTION", "SHAMPOO", "MAKEUP", "PERFUME", "MIRROR", "BRUSH", "COMB", "NAIL", 
        "HAIR", "LIPSTICK", "SERUM", "FACE MASK", "VITAMIN", "HERBAL", "SUPPLEMENT", "MEDICAL", 
        "MASSAGE"
    ],
    "Jewelry & Accessories": [
        "BRACELET", "NECKLACE", "RING", "EARRING", "WATCH", "KEYCHAIN", "BROOCH", "PIN", "CHARM", 
        "CHAIN", "PENDANT",  "GOLD", "SILVER"
    ],
    "Food & Beverages": [
        "CHOCOLATE", "CANDY", "SNACK", "TEA", "COFFEE", "BISCUIT", "JUICE", "WINE", "BEER", 
        "HONEY", "SPICES", "GIN", "TONIC", "BAR", "HERB", "FOOD"
    ],
    "Electronics & Gadgets": [
        "USB", "LAMP", "BATTERY", "WIRE", "PHONE", "CABLE", "CHARGER", "HEADPHONE", "LIGHTER", 
        "POWER", "SPEAKER", "LAPTOP", "TABLET", "FLASHLIGHT", "TORCH"
    ],
    "Bags & Pouches": [
        "BAG", "LUNCH", "PURSE", "PASSPORT"
    ],
    "Miscellaneous": [
        "SET", "OTHER", "VARIOUS", "ASSORTED", "RANDOM", "UNDEFINED", "POSTAGE", "STAMP", "MAIL", 
        "SAMPLES", "UNKNOWN", "MANUAL"
    ]
}


def classify_description(desc):
    desc_upper = desc.upper()
    for category, keywords in categories.items():
        if any(keyword in desc_upper for keyword in keywords):
            return category
    return "Miscellaneous"  


def process_data(df, output_path):
    df["Category"] = df["Description"].apply(classify_description)
    df.to_parquet(output_path, engine = 'pyarrow', index=False)
    return df

#
df_categorized = process_data(sampled_data, output_path)


In [17]:
df_categorized.query("Category == 'Miscellaneous'")

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Category
269594,560491,21917,SET 12 KIDS WHITE CHALK STICKS,1,2011-07-19 10:51:00,0.42,12415.0,Australia,Miscellaneous
193918,553546,21900,"KEY FOB , SHED",576,2011-05-17 15:42:00,0.53,12415.0,Australia,Miscellaneous
168402,551054,23050,RECYCLED ACAPULCO MAT GREEN,2,2011-04-26 11:44:00,8.25,12429.0,Austria,Miscellaneous
132653,547684,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-03-24 14:46:00,2.10,12408.0,Belgium,Miscellaneous
230459,557147,47518F,ICON PLACEMAT POP ART ELVIS,24,2011-06-17 10:51:00,0.12,12408.0,Belgium,Miscellaneous
...,...,...,...,...,...,...,...,...,...
282752,561658,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,1,2011-07-28 16:06:00,2.55,12743.0,Unspecified,Miscellaneous
323432,565303,22964,3 PIECE SPACEBOY COOKIE CUTTER SET,2,2011-09-02 12:17:00,2.10,NaN,Unspecified,Miscellaneous
498773,578539,23569,TRADTIONAL ALPHABET STAMP SET,4,2011-11-24 14:55:00,4.95,NaN,Unspecified,Miscellaneous
282797,561658,47348A,FUSCHIA VOILE POINTY SHOE DEC,1,2011-07-28 16:06:00,0.39,12743.0,Unspecified,Miscellaneous
